In [213]:
import numpy as np
import pandas as pd
import pickle as pkl

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import os
import time
import random

In [141]:
df = pd.read_csv('birdstrikes.csv', delimiter=',', low_memory=False)

In [142]:
bird_or_not = pd.DataFrame(columns = ['Species Name', 'bird_or_not'])
bird_or_not['Species Name'] = df['Species Name']
bird_or_not.head(5)

,Species Name,bird_or_not
0,GULL,NaN
1,HOUSE SPARROW,NaN
2,BARN OWL,NaN
3,UNKNOWN MEDIUM BIRD,NaN
4,FINCH,NaN


In [143]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [154]:
species_name = list(df['Species Name'].unique())
is_bird = []

In [162]:
google_url = 'https://www.google.com/'
driver = webdriver.Chrome(chromedriver)

from selenium.common.exceptions import NoSuchElementException

for species in species_name[679:]:
    if 'UNKNOWN' in species:
        is_bird.append('unknown')
    elif ',' in species:
        is_bird.append('multiple')
    else:
        driver.get(google_url)
        
        search = driver.find_element_by_id('lst-ib')
        search.send_keys(species)
        time.sleep(random.choice(range(1,3)))
        search.send_keys(Keys.RETURN)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        if soup.find('div', class_ = 'wwUB2c kno-fb-ctx') == None:
            is_bird.append('None')
        else:
            is_bird.append(soup.find('div', class_ = 'wwUB2c kno-fb-ctx').text)


In [166]:
len(species_name)

716

In [165]:
len(is_bird)

716

In [177]:
insert_is_bird = pd.DataFrame()
insert_is_bird['Species Name'] = species_name
insert_is_bird['is_bird'] = is_bird

with open('insert_is_bird.pkl', 'wb') as picklefile:
    pkl.dump(insert_is_bird, picklefile)

In [168]:
bird_classified_df = pd.merge(bird_or_not, insert_is_bird, on='Species Name')

In [174]:
#bird_classified_df.drop(columns='bird_or_not', inplace=True)
bird_classified_df.shape

(174104, 2)

In [172]:
with open('bird_classified.pkl', 'wb') as picklefile:
    pkl.dump(bird_classified_df, picklefile)

### Scrape more **bird weights** data to be able to categorize birds by size to use in model

In [337]:
with open('bird_df_clean.pkl', 'rb') as picklefile:
    df1 = pkl.load(picklefile)

In [338]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [346]:
SpeciesName = list(df1['Species Name'].unique())

In [391]:
#bird_length = []
#bird_weight = []
#bird_size = []

In [392]:
url = 'http://identify.whatbird.com/mwg/_/0/attrs.aspx'

from selenium.common.exceptions import NoSuchElementException

for sp in SpeciesName:
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    
    search = driver.find_element_by_name('ctl00$ContentPlaceHolder1$ctl00$TabbedContentContainer10$matches2$txtFilter')
    search.send_keys(sp)
    time.sleep(random.choice(range(0,3)))

    try:
        species_selector = '//a[contains(@href, "/overview/")]'
        driver.find_element_by_xpath(species_selector)
        
        driver.find_element_by_xpath(species_selector).click()

        driver.switch_to_window(driver.window_handles[1])

        id_selector = '//a[contains(@href, "/identification/")]'
        driver.find_element_by_xpath(id_selector).click()

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        li = soup.find_all('li')
        li = str(li)

        l = li.find('Length Range:')
        length = li[l:l+43]
        bird_length.append(length)

        w = li.find('Weight:')
        weight = li[w:w+34]
        bird_weight.append(weight)

        s = li.find('Size:')
        size = li[s:s+43]
        bird_size.append(size)
        
    except NoSuchElementException:
        bird_length.append('None')
        bird_weight.append('None')
        bird_size.append('None')


In [384]:
len(SpeciesName)

732

In [393]:
len(bird_length), len(bird_weight), len(bird_size)

(732, 732, 732)

In [400]:
bird_sizes_df = pd.DataFrame(columns = ['Species Name', 'Species Size', 'Species Length', 'Species Weight'])
bird_sizes_df['Species Name'] = SpeciesName
bird_sizes_df['Species Size'] = bird_size
bird_sizes_df['Species Length'] = bird_length
bird_sizes_df['Species Weight'] = bird_weight

bird_sizes_df.head()

,Species Name,Species Size,Species Length,Species Weight
0,GULL,Size: </span>\nLarge (16 - 32 in)\n\n,Length Range: </span>\n51 cm (20 in)\n\n,Weight: </span>\n567 g (20 oz)\n\n
1,HOUSE SPARROW,Size: </span>\nSmall (5 - 9 in)\n\n,Length Range: </span>\n14-17 cm (5.5-6.5 in),Weight: </span>\n39 g (1.37 oz)\n\n
2,BARN OWL,Size: </span>\nLarge (16 - 32 in)\n\n,Length Range: </span>\n36-51 cm (14-20 in)\n\n,Weight: </span>\n510 g (18 oz)\n\n
3,UNKNOWN MEDIUM BIRD,None,None,None
4,FINCH,Size: </span>\nSmall (5 - 9 in)\n\n,Length Range: </span>\n13 cm (5 in)\n\n,Weight: </span>\n14 g (0.5 oz)\n\n


In [401]:
with open('bird_sizes_df.pkl', 'wb') as picklefile:
    pkl.dump(bird_sizes_df ,picklefile)

In [91]:
cleaned_weights = [x.replace('\xa0', ' ') for x in bird_weights]
cleaned_weights

['None',
 '0.85 – 1.4 oz',
 '0.95 – 1.4 lbs',
 'unknown',
 'Lesser goldfinch: 0.28 – 0.41 oz',
 'None',
 '4.8 oz',
 '5.6 oz',
 '1.1 oz',
 '13 oz',
 'None',
 '0.85 – 1.4 oz',
 'Canada goose: 7.1 – 14 lbs',
 'unknown',
 '3.3 lbs',
 'None',
 'None',
 'None',
 '2 oz',
 '1.6 – 3.5 lbs',
 'None',
 'unknown',
 'None',
 'Canada goose: 7.1 – 14 lbs',
 'None',
 'None',
 '9.7 oz',
 '1.1 lbs',
 '4.2 oz',
 'None',
 'None',
 'None',
 '1.2 oz',
 '2 – 3.6 oz',
 '6.9 oz',
 '2 lbs',
 'None',
 'unknown',
 '7.8 oz',
 'None',
 'None',
 'None',
 '3 lbs',
 'None',
 '60,000 lbs',
 '2.6 – 4.3 lbs',
 'None',
 '1.5 oz',
 'None',
 '2.7 oz',
 '1.5 oz',
 'None',
 '1.4 – 1.7 oz',
 'None',
 'None',
 '2 – 3.6 oz',
 '2.8 oz',
 '0.7 – 1.4 lbs',
 'None',
 '4.3 oz',
 '0.7 – 1.4 lbs',
 '3.1 oz',
 'None',
 'None',
 'None',
 '0.56 oz',
 '5.8 lbs',
 '5.4 oz',
 '3.3 oz',
 'None',
 'None',
 '6.6 – 14 lbs',
 '3.1 lbs',
 '85.0 grams',
 'None',
 '0.63 – 0.81 oz',
 '3.5 oz',
 'None',
 'None',
 '4.1 oz',
 'Violet-green swallow: 0.5 

In [ ]:
cleaned_weights

In [139]:
match_weight = pd.DataFrame()
match_weight['Species Name'] = species_name
match_weight['Species Weight'] = cleaned_weights

match_weight[match_weight['Species Name'].str.contains(' BEAR')]

## NEED TO REMOVE: 'TURTLE', ' BAT', 'MAMMAL', 'UNKNOWN BIRD', 'RABBIT', ' F' 118, 'COMMON GRAY FOX', 'FOX SQUIRREL',
## REMOVE: 'ELK', 'HORSE', 'PORCUPINE', 'DOG', 'POND SLIDER', 'SKUNK', 'MINK', 'TORTOISE', 'BEAR'

,Species Name,Species Weight
600,AMERICAN BLACK BEAR,240 lbs


In [122]:
for sp in match_weight['Species Name']:
    print(sp)

GULL
HOUSE SPARROW
BARN OWL
UNKNOWN MEDIUM BIRD
FINCH
MUNIAS
PACIFIC GOLDEN-PLOVER
SPOTTED DOVE
HORNED LARK
ROCK PIGEON
WESTERN MEADOWLARK
SPARROW
GEESE
UNKNOWN BIRD
TURKEY VULTURE
HAWAIIAN DUCK
HERRING GULL
SANDPIPER
ZEBRA DOVE
MALLARD
SHORT-EARED OWL
UNKNOWN LARGE BIRD
DUCKS
 GEESE
 SWANS
HAWK
LAUGHING GULL
RING-BILLED GULL
MOURNING DOVE
DOUBLE-CRESTED CORMORANT
THRASHERS
 MOCKINGBIRDS
 CATBIRDS
EUROPEAN STARLING
MERLIN
WESTERN GULL
CANADA GOOSE
UNKNOWN SMALL BIRD
CATTLE EGRET
HERONS
 EGRETS
 BITTERNS
OSPREY
TOWNSEND'S SHEARWATER
BLACKBIRD
BLACK VULTURE
SHOREBIRD
SNOW BUNTING
DOVE
AMERICAN ROBIN
BROWN-HEADED COWBIRD
FRANKLIN'S GULL
ROSE-BREASTED GROSBEAK
DUCK
TERNS
STARLINGS
COMMON NIGHTHAWK
AMERICAN CROW
OWLS
WILSON'S SNIPE
CROW
KILLDEER
COMMON MYNA
SWALLOW
EGRET
SAVANNAH SPARROW
SNOW GOOSE
YELLOW-BILLED MAGPIE
EASTERN MEADOWLARK
RING-NECKED PHEASANT
NEW WORLD VULTURES
BALD EAGLE
GREAT FRIGATEBIRD
RED-WINGED BLACKBIRD
WREN
CAROLINA WREN
ARCTIC TERN
GULLS
 KITTIWAKES
AMERICAN KESTREL